<a href="https://colab.research.google.com/github/harikrishnareddymallavarapu/schoolofai/blob/master/Session5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytreebank
!pip install googletrans==3.1.0a0

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp37-none-any.whl size=37070 sha256=f6226625be2166ea9000a34d45c19c52d4f12e706af7a4a2c25420300b06b826
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank
     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 1.0MB 14.0MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=e75aa85de41e07d931b3b1a06b3665b33cbb59fa3b19a100fdd462e0587f56ef
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [ ]:
import pytreebank
dataset = pytreebank.load_sst()

In [ ]:
sentenceData_train = []
sentenceData_val = []
sentenceData_test = []

for sentenceNode in dataset["train"]:
   sentenceData_train.append(sentenceNode.to_labeled_lines()[0])

for sentenceNode in dataset["test"]:
   sentenceData_test.append(sentenceNode.to_labeled_lines()[0])

for sentenceNode in dataset["dev"]:
   sentenceData_val.append(sentenceNode.to_labeled_lines()[0])

In [ ]:
import pandas as pd
sentenceData_train = pd.DataFrame(sentenceData_train, columns=['labels','sentence'])
sentenceData_train.head()
sentenceData_train.reset_index(inplace=True)


sentenceData_test = pd.DataFrame(sentenceData_test, columns=['labels','sentence'])
sentenceData_test.head()

sentenceData_val = pd.DataFrame(sentenceData_val, columns=['labels','sentence'])
sentenceData_val.head()

,labels,sentence
0,3,It 's a lovely film with lovely performances b...
1,2,"No one goes unindicted here , which is probabl..."
2,3,And if you 're not nearly moved to tears by a ...
3,4,"A warm , funny , engaging film ."
4,4,Uses sharp humor and insight into human nature...


In [ ]:
sentenceData_train.shape, sentenceData_test.shape, sentenceData_val.shape

((8544, 3), (2210, 2), (1101, 2))

In [ ]:
import random
import googletrans
from googletrans import Translator

translator = Translator()
# sentence = ['']

# available_langs = list(googletrans.LANGUAGES.keys()) 
# trans_lang = random.choice(available_langs) 
# print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")
# print(translator.translate("This is so cool"))

# translations = translator.translate(["This is cool"], dest=trans_lang) 
# t_text = [t.text for t in translations]
# print(t_text)

# translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
# en_text = [t.text for t in translations_en_random]
# print(en_text)
from tqdm import tqdm

def translateData(sentenceList):
  available_langs = list(googletrans.LANGUAGES.keys())
  transList = []
  for sentence in tqdm(sentenceList):
  
    trans_lang = random.choice(available_langs)
    translations = translator.translate(sentence, dest=trans_lang) 
    
    translations_en_random = translator.translate(translations.text, src=trans_lang, dest='en') 

    transList.append(str(translations_en_random.text))
  return transList


In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    sentence = list(sentence)
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ''.join(sentence)

In [ ]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

Generating a random sample of 5000 augmented sentences using google translator

In [ ]:
k = random.sample(range(0, sentenceData_train.shape[0]), 6000)

text = translateData(sentenceData_train.sentence[k])
#creating augmented data
transSentences = pd.DataFrame(list(zip(sentenceData_train.labels[k], text)),
               columns =['labels', 'sentence'])
transSentences.head()
# sentenceData = sentenceData.append(transSentences)
# sentenceData.shape[0]

100%|██████████| 6000/6000 [1:13:41<00:00,  1.36it/s]


,labels,sentence
0,2,"Luxurious, excitingly tasteless."
1,2,But the players do not show this diss.
2,4,Jones provided a very interesting and touching...
3,4,A shocking and important film.
4,1,Nothing but one relentless depressing situatio...


In [ ]:
k = random.sample(range(0, sentenceData_train.shape[0]), 3000)
for i in k:
  sentenceData_train.sentence.loc[i] = random_swap(sentenceData_train.sentence[i],5)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
sentenceData_train = sentenceData_train.append(transSentences)
sentenceData_train.tail()

,index,labels,sentence
5995,NaN,1,"Sweet gentle Jesus , did the screenwriters jus..."
5996,NaN,4,"A colorful , vibrant introduction to a univers..."
5997,NaN,3,Marshall puts a suspenseful spin on standard h...
5998,NaN,0,"Apparently , romantic comedy with a fresh poin..."
5999,NaN,3,An alternately fascinating and frustrating doc...


In [ ]:
sentenceData_train.to_excel("stanfordData_train.xlsx", index=False)
sentenceData_test.to_excel("stanfordData_test.xlsx", index=False)
sentenceData_val.to_excel("stanfordData_val.xlsx", index=False)
sentenceData_train.shape

(14544, 3)

In [ ]:
sentenceData_train.to_excel("stanfordData_train_aug.xlsx")

In [ ]:
sentenceData_train = pd.read_excel("stanfordData_train_aug.xlsx")
sentenceData_test = pd.read_excel("stanfordData_test.xlsx")
sentenceData_val = pd.read_excel("stanfordData_val.xlsx")

In [ ]:
sentenceData = sentenceData_train.reset_index()

In [ ]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [ ]:
fields = [('sentence', sentence),('labels',Label)]

In [ ]:
example = [data.Example.fromlist([sentenceData.sentence.iloc[i],sentenceData.labels.iloc[i]], fields) for i in range(sentenceData.shape[0])] 
example2 = [data.Example.fromlist([sentenceData_val.sentence.iloc[i],sentenceData_val.labels.iloc[i]], fields) for i in range(sentenceData_val.shape[0])] 
example3 = [data.Example.fromlist([sentenceData_test.sentence.iloc[i],sentenceData_test.labels.iloc[i]], fields) for i in range(sentenceData_test.shape[0])] 

In [ ]:
train = data.Dataset(example, fields)
val = data.Dataset(example2, fields)
test = data.Dataset(example3, fields)

In [ ]:
(len(train), len(val), len(test))

(14544, 1101, 2210)

In [ ]:
sentence.build_vocab(train)
Label.build_vocab(train)

In [ ]:
print('Size of input vocab : ', len(sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  31658
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 13449), (',', 12168), ('the', 9673), ('a', 7291), ('of', 7176), ('and', 7060), ('to', 4889), ('-', 4586), ("'s", 4101), ('is', 4070)]
Labels :  defaultdict(None, {3: 0, 1: 1, 2: 2, 4: 3, 0: 4})


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, val, test), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(sentence.vocab.stoi, tokens)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(31658, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 9,739,505 trainable parameters


In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.sentence   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  

        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
       
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
from tqdm import tqdm
N_EPOCHS = 100
best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS)):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

In [ ]:
test_loss, test_acc